In [2]:
import pandas as pd
import numpy as np
from scipy.sparse import hstack

from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras.preprocessing.text import one_hot

Using TensorFlow backend.


In [12]:
with open('train-sampled.csv', 'a') as f:
    for i, df in enumerate(pd.read_csv('train', chunksize=500000)):
        df = pd.concat([df.loc[df['click'] == 0],
                        df.loc[df['click'] == 1].sample(frac=2, replace=True)])
        
        header = True if i == 0 else False
        df.sample(frac=1).to_csv(f,
                                 header=header,
                                 index=None)

In [17]:
d = {'C1':10, 'C14':3000,
     'C15':10, 'C16':10,
     'C17':500, 'C18':5,
     'C19':70, 'C20':180, 'C21':65,
     'site_id': 2000,
     'site_domain': 1000,
     'site_category': 50,
     'app_domain': 200,
     'app_category': 100,
     'device_type': 4,
     'device_id': 8000}

model = Sequential() 
model.add(Dense(units=2,
                input_dim=sum(d.values())+1,
                activation='sigmoid'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['sparse_categorical_accuracy'])

for df in pd.read_csv('train-sampled.csv',
                      chunksize=50000):
    X = np.hstack([to_categorical(df[col].apply(lambda r: (one_hot(str(r),
                                                                   n_features))
                                               ).tolist(),
                                  n_features)
                   for (col, n_features) in d.items()])

    X = np.hstack([X, df['banner_pos'].values.reshape(-1, 1)])
    
    model.fit(x=X,
              y=df['click'].values,
              epochs=7,
              validation_split=0.2)

Train on 40000 samples, validate on 10000 samples
Epoch 1/5
40000/40000 [==============================] - 32s - loss: 0.5417 - sparse_categorical_accuracy: 0.7410 - val_loss: 0.5307 - val_sparse_categorical_accuracy: 0.7424
Epoch 2/5
40000/40000 [==============================] - 25s - loss: 0.5211 - sparse_categorical_accuracy: 0.7488 - val_loss: 0.5266 - val_sparse_categorical_accuracy: 0.7427
Epoch 3/5
40000/40000 [==============================] - 32s - loss: 0.5147 - sparse_categorical_accuracy: 0.7523 - val_loss: 0.5247 - val_sparse_categorical_accuracy: 0.7424
Epoch 4/5
40000/40000 [==============================] - 26s - loss: 0.5100 - sparse_categorical_accuracy: 0.7536 - val_loss: 0.5242 - val_sparse_categorical_accuracy: 0.7441
Epoch 5/5
40000/40000 [==============================] - 18s - loss: 0.5062 - sparse_categorical_accuracy: 0.7547 - val_loss: 0.5235 - val_sparse_categorical_accuracy: 0.7445
